In [1]:

import numpy as np
from typing import Literal
import PIL.Image as Image
from glob import glob


import tensorflow as tf
from tqdm import tqdm, trange
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

from keras.utils import normalize
import os
import cv2
from matplotlib import pyplot as plt

In [2]:

"""
Standard Unet
Model not compiled here, instead will be done externally to make it
easy to test various loss functions and optimizers.
"""


from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda




################################################################
def multi_unet_model(n_classes=4, IMG_HEIGHT=256, IMG_WIDTH=256, IMG_CHANNELS=1):
#Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)

    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

    #Expansive path
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = Conv2D(n_classes, (1, 1), activation='softmax')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])

    #NOTE: Compile the model in the main program to make it easy to test with various loss functions
    #model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    #model.summary()

    return model

In [6]:
from google.colab import files
loading = files.upload()

Saving test(1).hdf5 to test(1).hdf5


In [4]:
!pip install patool
import patoolib
patoolib.extract_archive("/content/test_data.zip")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 2.7 MB/s eta 0:00:00


INFO patool: Extracting /content/test_data.zip ...
INFO:patool:Extracting /content/test_data.zip ...
INFO patool: running /usr/bin/7z x -o./Unpack_70v34voz -- /content/test_data.zip
INFO:patool:running /usr/bin/7z x -o./Unpack_70v34voz -- /content/test_data.zip
INFO patool:     with input=''
INFO:patool:    with input=''
INFO patool: ... /content/test_data.zip extracted to `test_data'.
INFO:patool:... /content/test_data.zip extracted to `test_data'.


'test_data'

In [5]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [7]:
def get_model():
    return multi_unet_model(n_classes=4, IMG_HEIGHT=256, IMG_WIDTH=256, IMG_CHANNELS=1)

model = get_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 1)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 256, 256, 16)         160       ['input_1[0][0]']             
                                                                                                  
 dropout (Dropout)           (None, 256, 256, 16)         0         ['conv2d[0][0]']              
                                                                                                  
 conv2d_1 (Conv2D)           (None, 256, 256, 16)         2320      ['dropout[0][0]']             
                                                                                              

In [8]:
model.load_weights("/content/test(1).hdf5")

In [ ]:
uploading = files.upload()

Saving 17gw5j.JPG to 17gw5j.JPG


In [89]:
import shutil
shutil.rmtree("/content/patches_with_padding")
shutil.rmtree("/content/segmented_patches")

Total patches created: 24


/content/patches_with_padding/patch_1001.png
/content/patches_with_padding/patch_1002.png
/content/patches_with_padding/patch_1003.png
/content/patches_with_padding/patch_1004.png
/content/patches_with_padding/patch_1005.png
/content/patches_with_padding/patch_1006.png
/content/patches_with_padding/patch_1007.png
/content/patches_with_padding/patch_1008.png
/content/patches_with_padding/patch_1009.png
/content/patches_with_padding/patch_1010.png
/content/patches_with_padding/patch_1011.png
/content/patches_with_padding/patch_1012.png
/content/patches_with_padding/patch_1013.png
/content/patches_with_padding/patch_1014.png
/content/patches_with_padding/patch_1015.png
/content/patches_with_padding/patch_1016.png
/content/patches_with_padding/patch_1017.png
/content/patches_with_padding/patch_1018.png
/content/patches_with_padding/patch_1019.png
/content/patches_with_padding/patch_1020.png
/content/patches_with_padding/patch_1021.png
/content/patches_with_padding/patch_1022.png
/content/p

(24, 256, 256)

In [ ]:
# import numpy as np
# from PIL import Image
# import os

# # Assuming your images are in a directory called 'images'
# image_dir = 'images'
# image_files = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.png')])

# # Initialize an empty array to hold the grayscale images
# num_images = len(image_files)
# image_shape = (256, 256)
# grayscale_images = np.empty((num_images, *image_shape), dtype=np.uint8)

# # Process each image
# for i, image_file in enumerate(image_files):
#     # Open the image
#     img = Image.open(image_file)

#     # Convert to grayscale
#     grayscale_img = img.convert('L')

#     # Convert to numpy array and store in the pre-allocated array
#     grayscale_images[i] = np.array(grayscale_img)

# # grayscale_images now holds your 24 grayscale images in the shape (24, 256, 256)
# print(grayscale_images.shape)


In [93]:
test_patches.shape

(24, 256, 256, 1)

1/1 [==============================] - 0s 145ms/step


(1024, 1536)
(1024, 1360)


In [23]:
uploading = files.upload()

Saving 17gw5j_gt.npy to 17gw5j_gt (1).npy


In [97]:
array = np.load("/content/17gw5j_gt.npy")
image = Image.fromarray(array*85)

image.save("hehe.png")

In [26]:
array = np.load("/content/17gw5j_gt.npy")
image = Image.fromarray(array*85)
image.save("hehe.png")
image.show()

In [107]:
from PIL import Image, ImageOps
import shutil
image_files = [f for f in os.listdir("/content/test_data/image") if f.endswith('.JPG')]
num_images = len(image_files)
print(num_images)
for i, image_file in enumerate(image_files):

    os.mkdir("patches_with_padding")
    # Load the original image
    image = Image.open(f"/content/test_data/image/{image_file}")
    # image = Image.open('/content/stranger-sections-2-train-data/label/17gw5j_gt.npy')


    # Original image dimensions
    original_width, original_height = image.size

    # New dimensions after padding
    new_width = 1536
    new_height = 1024

    # Calculate padding
    padding_left = (new_width - original_width) // 2
    padding_right = padding_left
    padding_top = 0
    padding_bottom = 0

    # Add padding to the image
    padded_image = ImageOps.expand(image, (padding_left, padding_top, padding_right, padding_bottom))

    # Dimensions of the padded image
    padded_width, padded_height = padded_image.size

    # Create patches
    patch_size = 256
    patches = []

    for y in range(0, padded_height, patch_size):
        for x in range(0, padded_width, patch_size):
            box = (x, y, x + patch_size, y + patch_size)
            patch = padded_image.crop(box)
            patches.append(patch)

    # Save patches or process further
    for i, patch in enumerate(patches):
        patch.save(f'patches_with_padding/patch_{i+1+1000}.png')

    print(f'Total patches created: {len(patches)}')

    test_patches = []
    import glob
    for directory_path in sorted(glob.glob("/content/patches_with_padding")):
        for img_path in sorted(glob.glob(os.path.join(directory_path, "*.png"))):
            print(img_path)
            img = cv2.imread(img_path, 0)
            #img = cv2.resize(img, (SIZE_Y, SIZE_X))
            test_patches.append(img)

    #Convert list to array for machine learning processing
    test_patches = np.array(test_patches)

    test_patches.shape
    test_patches = np.expand_dims(test_patches, axis=3)
    test_patches = normalize(test_patches, axis=1)

    os.makedirs("segmented_patches", exist_ok=True)
    # print(test_img.shape)


    for test_patch_number in range(0,24):
      test_img = test_patches[test_patch_number]
      # image = Image.open("/content/patches/patch_3.png")
      # test_img = np.array(image)
      # ground_truth=y_test[test_img_number]

      test_img_norm=test_img[:,:,0][:,:,None]
      test_img_input=np.expand_dims(test_img_norm, 0)

      prediction = (model.predict(test_img_input))
      predicted_img=np.argmax(prediction, axis=3)[0,:,:]
      filename = os.path.join("/content/segmented_patches", f'seg_patch{test_patch_number}')

      np.save(filename, predicted_img)

      # Number of patches per row and column
    patch_size = 256
    patches_per_row = 6  # 1536 // 256
    patches_per_column = 4  # 1024 // 256

    # Create an empty canvas with the dimensions of the original padded image
    new_width = patches_per_row * patch_size
    new_height = patches_per_column * patch_size
    canvas = np.zeros((new_height, new_width), dtype=np.uint8)

    # Load each patch from .npy files and place it in the correct position
    for i in range(patches_per_column):
        for j in range(patches_per_row):
            patch_index = i * patches_per_row + j
            patch = np.load(f'segmented_patches/seg_patch{patch_index}.npy')
            y_start = i * patch_size
            y_end = y_start + patch_size
            x_start = j * patch_size
            x_end = x_start + patch_size
            canvas[y_start:y_end, x_start:x_end] = patch

    # Define the dimensions to crop the canvas to the original unpadded size
    crop_left = 88
    crop_right = crop_left + 1360
    crop_top = 0
    crop_bottom = crop_top + 1024

    # Crop the canvas to remove the padding
    cropped_canvas = canvas[crop_top:crop_bottom, crop_left:crop_right]

    output_file = os.path.join("/content/result", f'{image_file[:6]}_pred.npy')
    np.save(output_file, cropped_canvas)


    # Convert the canvas to an image
    print(canvas.shape)
    print(cropped_canvas.shape)

    shutil.rmtree("/content/patches_with_padding")
    shutil.rmtree("/content/segmented_patches")


    # reconstructed_image = Image.fromarray(canvas*85)
    # reconstructed_image_cropped = Image.fromarray(cropped_canvas*85)
    # # Save or display the reconstructed image
    # reconstructed_image.save('reconstructed_image_segmented.png')
    # reconstructed_image_cropped.save('reconstructed_image_cropped_segmented.png')

    # reconstructed_image.show()

25
Total patches created: 24
/content/patches_with_padding/patch_1001.png
/content/patches_with_padding/patch_1002.png
/content/patches_with_padding/patch_1003.png
/content/patches_with_padding/patch_1004.png
/content/patches_with_padding/patch_1005.png
/content/patches_with_padding/patch_1006.png
/content/patches_with_padding/patch_1007.png
/content/patches_with_padding/patch_1008.png
/content/patches_with_padding/patch_1009.png
/content/patches_with_padding/patch_1010.png
/content/patches_with_padding/patch_1011.png
/content/patches_with_padding/patch_1012.png
/content/patches_with_padding/patch_1013.png
/content/patches_with_padding/patch_1014.png
/content/patches_with_padding/patch_1015.png
/content/patches_with_padding/patch_1016.png
/content/patches_with_padding/patch_1017.png
/content/patches_with_padding/patch_1018.png
/content/patches_with_padding/patch_1019.png
/content/patches_with_padding/patch_1020.png
/content/patches_with_padding/patch_1021.png
/content/patches_with_padd

In [108]:
import shutil

# Define the folder to be zipped and the name of the output zip file
folder_to_zip = '/content/result'  # Replace with the name of your folder
output_zip_file = 'result.zip'

# Create a zip archive of the folder
shutil.make_archive(output_zip_file.replace('.zip', ''), 'zip', folder_to_zip)

from google.colab import files
files.download(output_zip_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>